# Superstore Sales Exploratory Analysis

## Objective
The goal of this analysis is to explore a public retail sales dataset to identify revenue drivers, seasonal trends, and potential areas of underperformance. This project demonstrates a practical, end-to-end data analysis workflow using Python, Excel, and Tableau.

## Dataset
- Source: Public Superstore Sales dataset
- Data type: Transaction-level retail orders
- Key fields include order date, product category, sales, profit, and region

## Key Business Questions
- Which product categories and sub-categories drive the most revenue?
- How does sales performance vary over time?
- Which regions are underperforming in terms of profit?
- Are there high-sales but low-profit products?

## Tools
- Python (pandas, matplotlib, seaborn) for data cleaning and analysis
- Excel for pivot-table summaries
- Tableau for dashboard visualizations

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option("display.max_columns", None)

# Load dataset
df = pd.read_csv("../data/raw/superstore.csv")

# Preview data
df.head()

ModuleNotFoundError: No module named 'matplotlib'